In [348]:
import pandas as pd
import numpy as np
import scipy
from scipy.stats import norm
import random

In [349]:
# Price of Call Option function
def get_call( T, r, d, S0,K, sigma):
    if K==0 or T==0 or sigma ==0:
        print('Not a valid input')
    elif d == 0:
        d1 = (np.log(S0/K)+(r+0.5*sigma**2)*T) / (sigma*np.sqrt(T))
        d2 = d1-sigma*np.sqrt(T)
        call = S0*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
        return call
    elif d < 0:
        print('Not a valid dividend input')
    else:
        d1 = (np.log(S0/K)+(r+0.5*sigma**2)*T) / (sigma*np.sqrt(T))
        d2 = d1-sigma*np.sqrt(T)
        print(d1)
        call = np.exp(-d*T)*S0*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
        return(call)

In [350]:
# BS Delta of a call option
def get_delta_call(T,r,d,S0,K,sigma):
    d1 = (np.log(S0/K)+(r+0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1-sigma*np.sqrt(T)
    delta= norm.cdf(d1)
    return delta , d1

In [360]:
tau =[1.00,0.92,0.83,0.75,0.67,0.58,0.50,0.42,0.33,0.25,0.17,0.08,0.000001]
z=[0]
for i in range (0,12):
    z.append(np.random.randn())
S=[100]
periods = 12
r=0
K=106
T=1
sigma=0.17
C0=get_call(T=1, r=0, d=0, S0=100,K=106, sigma=0.17)
d1=[]
delta=[]
ds=[]
portfolio=[]
bs=[]
pnl=[]

#generate random stock process
S= [100 ,S[0]*np.exp((r-0.5*sigma**2)/periods + sigma*z[2]/np.sqrt(periods))]
for j in range(1,12):
    S.append(S[j]*np.exp((r-0.5*sigma**2)/periods + sigma*z[j+1]/np.sqrt(periods)))


#Generate d1 column
for q in range(0,13):
    d1.append((np.log(S[q]/K)+(r+0.5*sigma**2)*tau[q]) / (sigma*np.sqrt(tau[q])))
    
    delta.append(norm.cdf(d1[q]))
    ds.append(delta[q]*S[q])


#Generate cash and portoflio columns   
cash=[C0-ds[0]]

for c in range(0,12):
    cash.append(cash[c]-(delta[c+1]-delta[c])*S[c+1])
  

#generate BS Call price evolution
for u in range(0,len(cash)):
    portfolio.append(ds[u]+cash[u])
    bs.append(get_call(tau[u], 0, 0, S[u],106, 0.17))

for p in range(0,13):
    pnl.append(portfolio[p]-bs[p])
    
final_pnl=ds[-1]+cash[-1]-(np.maximum(S[-1]-K,0))
    


s = {'S':S,'d1':d1,'delta':delta,'ds':ds,'cash':cash,'port':portfolio,'bs':bs,'pnl':pnl}
pd.DataFrame(s)




,S,d1,delta,ds,cash,port,bs,pnl
0,100.00,-0.26,0.40,39.83,-35.45,4.38,4.38,0.00
1,107.28,0.16,0.56,60.26,-52.98,7.28,7.59,-0.31
2,115.10,0.61,0.73,83.88,-72.20,11.67,12.31,-0.64
3,114.88,0.62,0.73,84.12,-72.61,11.51,11.86,-0.35
4,114.90,0.65,0.74,85.24,-73.71,11.53,11.58,-0.05
5,111.94,0.49,0.69,76.86,-67.52,9.34,9.09,0.25
6,108.29,0.24,0.59,64.34,-57.51,6.83,6.36,0.47
7,101.61,-0.33,0.37,37.70,-34.84,2.86,2.70,0.16
8,112.60,0.67,0.75,84.19,-77.25,6.93,8.34,-1.41
9,113.79,0.88,0.81,92.14,-84.32,7.83,8.85,-1.02
